# Instrumental Variables

This notebook will simulate IV with bianry treatment and instrument.

## Import packages

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pathlib
import matplotlib.pyplot as plt

## Set up directory structure

In [4]:
repo_f = pathlib.Path.cwd().parent

## Simple IV with Constant Treatment Effects

In [38]:
def create_IV_data(coef_interest=0.8, epsilon_1_and_2=0.5):

    # Create a dataframe 

    # Main independent variable
    df = pd.DataFrame(np.random.normal(0, 1, 100000).reshape(100000, 1), columns=['epsilon_2'])

    df['X_2'] = np.random.normal(0, 2, 100000) + df['epsilon_2']

    # Create instrument
    df['Z'] = np.random.uniform(0, 1, 100000)

    # Round to make Z binary
    df['Z'] = np.round(df['Z'])

    # Treatment variable that is 1 if latent variable is greater than 0
    # Notice that the instrument only affects the treatment variable in a
    # positive way
    
    df['treatment'] = np.where(df['X_2'] + df['Z'] > 0, 1, 0)

    # Error term
    df['epsilon_1'] = epsilon_1_and_2*df['epsilon_2']

    # Dependent variable
    df['Y'] = coef_interest*df['treatment'] + 5 + df['epsilon_1']
    

    return df


In [39]:
df = create_IV_data()

In [40]:
# Run OLS regression
X = df[['treatment']]
X = sm.add_constant(X)
Y = df['Y']
model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.593
Method:                 Least Squares   F-statistic:                 1.457e+05
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:09:19   Log-Likelihood:                -66105.
No. Observations:              100000   AIC:                         1.322e+05
Df Residuals:                   99998   BIC:                         1.322e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.7935      0.002   2078.689      0.0

In [41]:
# Run IV regression "by hand"

# First stage
X = df[['Z']]
X = sm.add_constant(X)
Y = df['treatment']
model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

# Second stage
X = df[['Z']]
X = sm.add_constant(X)
Y = df['Y']
model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              treatment   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     3330.
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:09:24   Log-Likelihood:                -69405.
No. Observations:              100000   AIC:                         1.388e+05
Df Residuals:                   99998   BIC:                         1.388e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4986      0.002    230.215      0.0

In [42]:
# Run IV regression using statsmodels
import statsmodels.sandbox.regression.gmm as gmm
Z = df[['treatment','Z']]
Z = sm.add_constant(Z)
Y = df['Y']
model = gmm.IV2SLS(endog = Y, exog = Z[['treatment', 'const']], instrument = Z[['Z', 'const']])
results = model.fit()
print(results.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                      Y   R-squared:                       0.534
Model:                         IV2SLS   Adj. R-squared:                  0.534
Method:                     Two Stage   F-statistic:                     1915.
                        Least Squares   Prob (F-statistic):               0.00
Date:                Tue, 18 Apr 2023                                         
Time:                        10:10:01                                         
No. Observations:              100000                                         
Df Residuals:                   99998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
treatment      0.7854      0.018     43.762      0.0

## Violate exclusion

In [111]:
def create_IV_data(coef_interest=0.8, epsilon_1_and_2=0.5, violate_exclusion=False):

    # Create a dataframe 

    # Main independent variable
    df = pd.DataFrame(np.random.normal(0, 1, 100000).reshape(100000, 1), columns=['epsilon_2'])

    df['X_2'] = np.random.normal(0, 2, 100000) + df['epsilon_2']

    # Create instrument
    df['Z'] = np.random.uniform(0, 1, 100000)

    # Round to make Z binary
    df['Z'] = np.round(df['Z'])

    # Treatment variable that is 1 if latent variable is greater than 0
    # Notice that the instrument only affects the treatment variable in a
    # positive way
    
    df['treatment'] = np.where(df['X_2'] + df['Z'] > 0, 1, 0)

    # Error term
    df['epsilon_1'] = epsilon_1_and_2*df['epsilon_2']

    # Dependent variable
    if violate_exclusion:
        df['Y'] = coef_interest*df['treatment'] + 5 + df['epsilon_1'] + df['Z']
    else:
        df['Y'] = coef_interest*df['treatment'] + 5 + df['epsilon_1']
        
    return df


In [112]:
df_violate = create_IV_data(violate_exclusion=True)

In [113]:
# Run OLS regression
X = df_violate[['treatment']]
X = sm.add_constant(X)
Y = df_violate['Y']
model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.498
Model:                            OLS   Adj. R-squared:                  0.498
Method:                 Least Squares   F-statistic:                 9.933e+04
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:53:55   Log-Likelihood:                -99816.
No. Observations:              100000   AIC:                         1.996e+05
Df Residuals:                   99998   BIC:                         1.997e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1913      0.003   1608.084      0.0

In [115]:
# Run IV regression using statsmodels
import statsmodels.sandbox.regression.gmm as gmm
Z = df_violate[['treatment','Z']]
Z = sm.add_constant(Z)
Y = df_violate['Y']
model = gmm.IV2SLS(endog = Y, exog = Z[['treatment', 'const']], instrument = Z[['Z', 'const']])
results = model.fit()
print(results.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                      Y   R-squared:                      -7.120
Model:                         IV2SLS   Adj. R-squared:                 -7.120
Method:                     Two Stage   F-statistic:                     4636.
                        Least Squares   Prob (F-statistic):               0.00
Date:                Tue, 18 Apr 2023                                         
Time:                        10:54:26                                         
No. Observations:              100000                                         
Df Residuals:                   99998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
treatment      6.5237      0.096     68.087      0.0